# Data collected from OP's website op-koti.fi. Initially I used webscraping technique to collect data. But using the API was more dynamic and easy to handle

In [2]:
import numpy as np
import pandas as pd
import requests
# import math
import re
from pandas import json_normalize

pd.options.display.max_columns = None
# display(df)

In [3]:
URL = 'https://op-koti.fi/api/apartments?mode=sale&featureGroup=apartment&orderBy=created&order=desc&offset=0&limit=4500'
# request the URL and parse the JSON
response = requests.get(URL)
response.raise_for_status() # raise exception if invalid response
data = response.json()['listings']

# normalizing the json data into a dataframe
df = json_normalize(data, max_level = 1)

In [4]:
df.head(10)

,id,kipinaId,listingType,commission,listingGroup,source,created,featureGroups,floor,year,imageUrl,showings,siteArea,rooms,numberOfRooms,price,debtFreePrice,rentPerMonth,location.streetAddress,location.postalCode,location.postOffice,location.city,location.cityId,location.region,location.regionId,location.district,location.districtId,location.streetId,location.coordinates,housing.livingArea,housing.totalArea,housing.rooms,housing.numberOfRooms,pricing.price,pricing.debtFreePrice,livingArea.size,livingArea.displayUnit,totalArea.size,totalArea.displayUnit,housing.siteArea,siteArea.size,siteArea.displayUnit,virtualPresentationUrl,livingArea,totalArea,newHousing.newHousingId,newHousing.name,videoUrl
0,624174,353942,91,O,82,Kipinä,2021-07-12T10:10:38.763Z,[apartment],0.0,1978.0,https://op-koti.fi/kuvat/624174/a93bfb3a065f39...,[],NaN,"2h,k,s+autotalli",2.0,42900.0,42900.0,None,Köyliöntie 47 as,32810,PEIPOHJA,Kokemäki,271,Kokemäki,188,Peipohja,3986.0,5061ab877609deb48f5f163d6e0c807bb4925f09,"{'latitude': 61.25273, 'longitude': 22.29649}","{'size': 60, 'displayUnit': 'm2'}","{'size': 60, 'displayUnit': 'm2'}","2h,k,s+autotalli",2.0,42900.0,42900.0,60.0,m2,60.0,m2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,510868,435577,90,K,82,Kipinä,2021-07-12T10:10:35.769Z,[apartment],0.0,2007.0,https://op-koti.fi/kuvat/510868/9e7e8048813fba...,[],NaN,"4h,k, kph, s, wc",4.0,248000.0,NaN,None,Kivilinnantie 135,76100,PIEKSÄMÄKI,Pieksämäki,593,Pieksämäki,314,Kontiomäki,5817.0,4a7474532e811b3c90bc2e5eb63545b0b1b1c31f,NaN,"{'size': 102, 'displayUnit': 'm2'}","{'size': 160.4, 'displayUnit': 'm2'}","4h,k, kph, s, wc",4.0,248000.0,NaN,102.0,m2,160.4,m2,"{'size': 6300, 'displayUnit': 'm2'}",6300.0,m2,NaN,NaN,NaN,NaN,NaN,NaN
2,510613,435321,90,K,82,Kipinä,2021-07-12T10:09:50.211Z,[apartment],0.0,1983.0,https://op-koti.fi/kuvat/510613/2c818af748bb3c...,[],NaN,"4h,k,khh,kph,s,wc+kellari,at/var",4.0,160000.0,NaN,None,Puomitie 7,96900,SAARENKYLÄ,Rovaniemi,698,Rovaniemi,358,Saarenkylä,5915.0,3aa5e565f4e55a2b70d248f9ffed2049b548865f,"{'latitude': 66.51804, 'longitude': 25.81208}","{'size': 120, 'displayUnit': 'm2'}","{'size': 148, 'displayUnit': 'm2'}","4h,k,khh,kph,s,wc+kellari,at/var",4.0,160000.0,NaN,120.0,m2,148.0,m2,"{'size': 2294, 'displayUnit': 'm2'}",2294.0,m2,NaN,NaN,NaN,NaN,NaN,NaN
3,510046,434754,89,O,82,Kipinä,2021-07-12T10:09:22.172Z,[apartment],6.0,1976.0,https://op-koti.fi/kuvat/510046/8248aee1737637...,"[{'showingType': 'unknown', 'date': '15.7.', '...",NaN,1h + kk,1.0,169000.0,169000.0,None,Asemapäällikönkatu 3 A,00520,HELSINKI,Helsinki,91,Etelä-Helsinki,44,Pasila,6686.0,82310caf895252a00d4fd78bbe5075b403ed7752,NaN,"{'size': 27.5, 'displayUnit': 'm2'}","{'size': 27.5, 'displayUnit': 'm2'}",1h + kk,1.0,169000.0,169000.0,27.5,m2,27.5,m2,"{'size': 2916, 'displayUnit': 'm2'}",2916.0,m2,NaN,NaN,NaN,NaN,NaN,NaN
4,502653,427366,91,O,82,Kipinä,2021-07-12T10:06:44.201Z,[apartment],1.0,1977.0,https://op-koti.fi/kuvat/502653/f70946996ae4ec...,[],NaN,"6h,k",6.0,189000.0,189000.0,None,Retkeilijäntie 18 B,70200,KUOPIO,Kuopio,297,Kuopio,207,Puijonlaakso,3029.0,71017f588d0afa880294c046f566ee55cf6b3fac,"{'latitude': 62.90407, 'longitude': 27.62824}","{'size': 143, 'displayUnit': 'm2'}","{'size': 154, 'displayUnit': 'm2'}","6h,k",6.0,189000.0,189000.0,143.0,m2,154.0,m2,"{'size': 9553.2, 'displayUnit': 'm2'}",9553.2,m2,NaN,NaN,NaN,NaN,NaN,NaN
5,501085,425803,90,K,82,Kipinä,2021-07-12T10:05:45.451Z,[apartment],0.0,1965.0,https://op-koti.fi/kuvat/501085/278cf1bb9e2a2b...,[],NaN,"5h, k, khh, kph, 3 wc, 2 vh, s, at",5.0,150000.0,NaN,None,Sirkankatu 10,18100,HEINOLA,Heinola,111,Heinola,39,Pyhätön,369.0,380c135e404a61a2f79ddb87e603375181827f2c,"{'latitude': 61.20997, 'longitude': 26.02726}","{'size': 155, 'displayUnit': 'm2'}","{'size': 235, 'displayUnit': 'm2'}","5h, k, khh, kph, 3 wc, 2 vh, s, at",5.0,150000.0,NaN,155.0,m2,235.0,m2,"{'size': 1101, 'displayUnit': 'm2'}",1101.0,m2,NaN,NaN,NaN,NaN,NaN,NaN
6,501165,425883,89,O,82,Kipinä,2021-07-12T10:05:17.881Z,[apartment],2.0,195

In [5]:
# df[df.id == '696611']
# display(df)

It can be seen above that the dataframe contains multiple columns that are irrelevant to us. Relevant columns are selected as shown below.

In [6]:
df.columns

Index(['id', 'kipinaId', 'listingType', 'commission', 'listingGroup', 'source',
       'created', 'featureGroups', 'floor', 'year', 'imageUrl', 'showings',
       'siteArea', 'rooms', 'numberOfRooms', 'price', 'debtFreePrice',
       'rentPerMonth', 'location.streetAddress', 'location.postalCode',
       'location.postOffice', 'location.city', 'location.cityId',
       'location.region', 'location.regionId', 'location.district',
       'location.districtId', 'location.streetId', 'location.coordinates',
       'housing.livingArea', 'housing.totalArea', 'housing.rooms',
       'housing.numberOfRooms', 'pricing.price', 'pricing.debtFreePrice',
       'livingArea.size', 'livingArea.displayUnit', 'totalArea.size',
       'totalArea.displayUnit', 'housing.siteArea', 'siteArea.size',
       'siteArea.displayUnit', 'virtualPresentationUrl', 'livingArea',
       'totalArea', 'newHousing.newHousingId', 'newHousing.name', 'videoUrl'],
      dtype='object')

In [7]:
cols = ['id', 'listingType', 'floor', 'year', 'rooms', 'numberOfRooms', 'price', 'debtFreePrice',
       'location.city', 'location.region', 'location.district','location.postalCode',
       'livingArea.size', 'totalArea.size']

df = df[cols]

Renaming the cols

In [8]:
df = df.rename(columns = {'location.city':'city','location.region':'region','location.district':'district','location.postalCode':'postalCode','livingArea.size':'livingArea','totalArea.size':'totalArea', 'year':'yearBuilt'})

Mapping the listingType column to the apporpriate type of house (extracted from the website manually)

In [9]:
lType = {89:'Kerrostalo',90:'Omakotitalo',91:'Rivitalo',92:'Paritalo',93:'Erillistalo',112:'Puutalo',113:'Luhtitalo',470:'Kytketty paritalo'}
df['listingType'] = df['listingType'].map(lType)

In [10]:
# some values in df.district are ''. So i decided to fill them with np.nan and then remove them altogether in the next step
df[df.district == ''] = np.nan

In [11]:
def count_NAs(df):
    for col in df:
        print(col," : ",df[col].isnull().sum())
        
# def homeSizeLabels(area):
#     """these ranges were calulated by using pd.cut(). 1=Smallest...4=Largest"""
#     if area in range(0,100):
#         return 1
#     elif area in range(100,175):
#         return 2
#     elif area in range(175,250):
#         return 3
#     else:
#         return 4 

In [12]:
# df = df.dropna(inplace = True)
count_NAs(df)

id  :  235
listingType  :  235
floor  :  678
yearBuilt  :  246
rooms  :  360
numberOfRooms  :  360
price  :  360
debtFreePrice  :  1608
city  :  235
region  :  235
district  :  235
postalCode  :  235
livingArea  :  360
totalArea  :  360


There are some columns with null values for debtFreePrice. Those values are replaced by price. 

In [13]:
df.loc[df.debtFreePrice.isnull() & df.price.notnull(),['debtFreePrice']] = df.price

Converting the columns to string and integer(from float) in order to remove the .0 at the end of figures

In [14]:
# df['id'] = df['id'].astype(str)
# df['floor'] = df['floor'].astype(int)
# df['numberOfRooms'] = df['numberOfRooms'].astype(int)
# df['yearBuilt'] = df['yearBuilt'].astype(int)
# df['postalCode'] = df['postalCode'].astype(str)

Calculating building age from 'yearBuilt'. Some buildings are supposed to be completed in the future. So any building that gets completed in the future year are also grouped into '1 year old buildings'

In [15]:
df['buildingAge'] = [1 if x > 2021 else (pd.to_datetime('today').year - pd.to_datetime(x, format = "%Y").year) for x in df.yearBuilt]
# df['centrum'] = [1 if x == 'Keskusta' else 0 for x in df.district]
# df['howBig'] = [homeSizeLabels(x) for x in df.livingArea]

In [16]:
# hideRows = ['listingGroup', 'yearBuilt', 'imageUrl', 'streetAddress', 'postalCode', 'region', 'link']
df.drop('yearBuilt', axis = 1, inplace = True)
df.reset_index(drop = True)

,id,listingType,floor,rooms,numberOfRooms,price,debtFreePrice,city,region,district,postalCode,livingArea,totalArea,buildingAge
0,624174,Rivitalo,0.0,"2h,k,s+autotalli",2.0,42900.0,42900.0,Kokemäki,Kokemäki,Peipohja,32810,60.0,60.0,43.0
1,510868,Omakotitalo,0.0,"4h,k, kph, s, wc",4.0,248000.0,248000.0,Pieksämäki,Pieksämäki,Kontiomäki,76100,102.0,160.4,14.0
2,510613,Omakotitalo,0.0,"4h,k,khh,kph,s,wc+kellari,at/var",4.0,160000.0,160000.0,Rovaniemi,Rovaniemi,Saarenkylä,96900,120.0,148.0,38.0
3,510046,Kerrostalo,6.0,1h + kk,1.0,169000.0,169000.0,Helsinki,Etelä-Helsinki,Pasila,00520,27.5,27.5,45.0
4,502653,Rivitalo,1.0,"6h,k",6.0,189000.0,189000.0,Kuopio,Kuopio,Puijonlaakso,70200,143.0,154.0,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4009,604375,Kerrostalo,1.0,"1h,k",1.0,18500.0,18500.0,Virrat,Virrat,Nallela,34800,30.0,30.0,51.0
4010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4011,592934,Rivitalo,1.0,"2h, k",2.0,126000.0,126000.0,Rauma,Rauma,Ota,26100,81.0,81.0,85.0
4012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The values in the 'rooms' column are separated using , or + or . or * and need to be cleaned

In [17]:
df.dropna(inplace = True)
count_NAs(df)

id  :  0
listingType  :  0
floor  :  0
rooms  :  0
numberOfRooms  :  0
price  :  0
debtFreePrice  :  0
city  :  0
region  :  0
district  :  0
postalCode  :  0
livingArea  :  0
totalArea  :  0
buildingAge  :  0


In [18]:
def clean_rooms(item):
    delimiters = "+", ","
    # >>> example = "stackoverflow (c) is awesome... isn't it?"
    regexPattern = '|'.join(map(re.escape, delimiters))
    return([re.sub('\\+|,|\\.|\\*| ', '', x) for x in re.split(regexPattern, item)])

df.rooms = [clean_rooms(x) for x in df.rooms]

These functions take a single entry from df.rooms and detects the features. These features are then represented with new columns using 1 or 0 denoting the presence or absence of the feature

In [19]:
def detect_centrum(postalCode):
    if postalCode[2:] == '100':
        return 1
    else:
        return 0

df['centrum'] = [detect_centrum(x) for x in df.postalCode]

def detect_sauna(room): #v4
    if 's' in room or str(room).find('sauna') != -1:
        return 1
    else:
        return 0
df['hasSauna'] = [detect_sauna(x) for x in df.rooms]

def detect_balcony(room): #v3
    if 'p' in room or str(room).find('parv') != -1:
        return 1
    else:
        return 0
df['hasBalcony'] = [detect_balcony(x) for x in df.rooms]

def detect_parking(room): #v3
    if 'ak' in room or 'at' in room or str(room).find('auto') != -1:
        return 1
    else:
        return 0
df['hasParking'] = [detect_parking(x) for x in df.rooms]

def detect_walk_in_closet(room): #v3
    if 'v' in room or 'vh' in room or str(room).find('vaate') != -1:
        return 1
    else:
        return 0
df['hasWalkInCloset'] = [detect_walk_in_closet(x) for x in df.rooms]

def detect_storage_room(room):
    if str(room).find('var') != -1:
        return 1
    else:
        return 0
df['hasStorageRoom'] = [detect_storage_room(x) for x in df.rooms]

In [20]:
df.drop('rooms', axis = 1, inplace = True)

In [21]:
df['id'] = df['id'].astype(str)
df['floor'] = df['floor'].astype(int)
df['numberOfRooms'] = df['numberOfRooms'].astype(int)
df['postalCode'] = df['postalCode'].astype(str)

In [22]:
df.head()

,id,listingType,floor,numberOfRooms,price,debtFreePrice,city,region,district,postalCode,livingArea,totalArea,buildingAge,centrum,hasSauna,hasBalcony,hasParking,hasWalkInCloset,hasStorageRoom
0,624174,Rivitalo,0,2,42900.0,42900.0,Kokemäki,Kokemäki,Peipohja,32810,60.0,60.0,43.0,0,1,0,1,0,0
1,510868,Omakotitalo,0,4,248000.0,248000.0,Pieksämäki,Pieksämäki,Kontiomäki,76100,102.0,160.4,14.0,1,1,0,0,0,0
2,510613,Omakotitalo,0,4,160000.0,160000.0,Rovaniemi,Rovaniemi,Saarenkylä,96900,120.0,148.0,38.0,0,1,0,0,0,1
3,510046,Kerrostalo,6,1,169000.0,169000.0,Helsinki,Etelä-Helsinki,Pasila,00520,27.5,27.5,45.0,0,0,0,0,0,0
4,502653,Rivitalo,1,6,189000.0,189000.0,Kuopio,Kuopio,Puijonlaakso,70200,143.0,154.0,44.0,0,0,0,0,0,0


In [23]:
set(df.listingType)

{'Erillistalo',
 'Kerrostalo',
 'Kytketty paritalo',
 'Luhtitalo',
 'Omakotitalo',
 'Paritalo',
 'Puutalo',
 'Rivitalo'}

Exporting the dataframe to a csv file

In [24]:
df.to_csv(f'op-koti.csv', sep=',', encoding='utf-8', index=False)